In [1]:
import pandas as pd
import re
file_name = input('Enter fine name:')
try:
    df = pd.read_csv(file_name+'.txt', header=None, sep =' ')
    print(f'Successfully opened {file_name}')
    print(df.head())
except:
    print('File cannot be found.')

Enter fine name: class1


Successfully opened class1
                                                   0
0  N00000001,A,A,D,D,C,D,D,A,,C,D,B,C,,B,C,B,D,A,...
1  N00000002,,A,,D,,B,D,A,C,C,D,,A,A,A,C,B,D,C,C,...
2  N00000003,B,A,,D,C,B,D,A,C,C,,B,A,B,A,C,B,D,A,...
3  N00000004,B,B,D,,,B,D,A,C,C,D,B,A,B,A,C,B,D,,C...
4  N00000005,B,A,,D,,B,D,A,C,C,D,B,A,B,A,C,B,D,D,...


In [2]:
df_error1 = df[df[0].apply(lambda x: len(x.split(',')) != 26)]
df_error1.head()

,0


In [3]:
def error2(x):
    return len(re.findall('N[0-9]{8,8}', x)) == 0
df_error2 = df[df[0].apply(error2)]
df_error2.head()

,0


In [4]:
df_valid = df.drop(df_error1.index.append(df_error2.index))
df_valid.shape

(20, 1)

In [5]:
#Analyzing
print('**** ANALYZING ****')
if len(df_error1) + len(df_error2) == 0:
    print('No errors found!')
else: 
    print('Invalid line of data: does not contain exactly 26 values:')
    for i in df_error1.index:
        print(df_error1.loc[i,0])
    print('Invalid line of data: N# is invalid')
    for i in df_error2.index:
        print(df_error2.loc[i,0])
#Report
print('**** REPORT ****')
print(f'Total lines of data: {len(df)}')
print(f'Total valid lines of data: {len(df_valid)}')
print(f'Total invalid lines of data: {len(df_error1) + len(df_error2)}')

**** ANALYZING ****
No errors found!
**** REPORT ****
Total lines of data: 20
Total valid lines of data: 20
Total invalid lines of data: 0


In [6]:
df1 = df_valid[0].str.split(',', expand=True, n=-1)
df1.rename(columns = {0:'id'}, inplace=True)
df1 = df1.set_index('id')
df1.head()

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
id,,,,,,,,,,,,,,,,,,,,,
N00000001,A,A,D,D,C,D,D,A,,C,...,C,B,D,A,C,,A,,C,D
N00000002,,A,,D,,B,D,A,C,C,...,C,B,D,C,C,A,A,B,,D
N00000003,B,A,,D,C,B,D,A,C,C,...,C,B,D,A,,A,,B,D,D
N00000004,B,B,D,,,B,D,A,C,C,...,C,B,D,,C,A,D,B,C,D
N00000005,B,A,,D,,B,D,A,C,C,...,C,B,D,D,C,A,A,,D,D


In [7]:
answer_key = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
key = answer_key.split(',')
for i in df1.index:
    for y in df1.columns:
        if df1.loc[i,y] == '': df1.loc[i,y] = 0
        elif df1.loc[i,y] == key[y-1]: df1.loc[i,y] = 4
        elif df1.loc[i,y] != key[y-1]: df1.loc[i,y] = -1
df1.head()

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
id,,,,,,,,,,,,,,,,,,,,,
N00000001,-1,4,4,4,4,-1,4,4,0,4,...,4,4,4,4,4,0,4,0,-1,4
N00000002,0,4,0,4,0,4,4,4,4,4,...,4,4,4,-1,4,4,4,4,0,4
N00000003,4,4,0,4,4,4,4,4,4,4,...,4,4,4,4,0,4,0,4,4,4
N00000004,4,-1,4,0,0,4,4,4,4,4,...,4,4,4,0,4,4,-1,4,-1,4
N00000005,4,4,0,4,0,4,4,4,4,4,...,4,4,4,-1,4,4,4,0,4,4


In [8]:
df_skip = df1.apply(lambda x: x==0, axis=1).sum()
df_wrong = df1.apply(lambda x: x==-1, axis=1).sum()

In [9]:
df2 = pd.DataFrame()
df2['score'] = df1.sum(axis=1)
print('Total student of high scores: {}'.format(len(df2[df2['score']>80])))
print('Mean (average) score: {}'.format(df2['score'].mean()))
print('Highest score: {}'.format(df2['score'].max()))
print('Lowest score: {}'.format(df2['score'].min()))
print('Range of scores: {}'.format(df2['score'].max()-df2.min()))
print('Median score: {}'.format(df2['score'].median()))
print(f'Question that most people skip: {list(df_skip[df_skip == df_skip.max()].index)}-{df_skip.max()}-{round(df_skip.max()/len(df_valid),2)}')
print(f'Question that most people wrong: {list(df_wrong[df_wrong == df_wrong.max()].index)}-{df_wrong.max()}-{round(df_wrong.max()/len(df_valid),2)}')

Total student of high scores: 6
Mean (average) score: 75.6
Highest score: 91.0
Lowest score: 59.0
Range of scores: score    32.0
dtype: float64
Median score: 73.0
Question that most people skip: [3, 5, 23]-4-0.2
Question that most people wrong: [10, 14, 16, 19, 22]-4-0.2


In [10]:
df2.head()

,score
id,
N00000001,59.0
N00000002,70.0
N00000003,84.0
N00000004,73.0
N00000005,83.0


In [11]:
df2.to_csv(file_name+'_grades.txt')